In [1]:
import pandas as pd
import re
import spacy
import functions

c:\Users\joseph\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dmk = pd.read_excel('./[unfiltered]greek_news_by_paragraphs.xlsx')

In [8]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

### With custom function ~ 90%

In [15]:
dmk_sentences = []
for idx,row in dmk.iterrows():
    for sentence in split_into_sentences(row['paragraph']):
        if len(sentence.split())>2: #just in case any symbol or punctuation was not filtered
            dmk_sentences.append({'title':row['title'], 'date':row['date'], 'sentence':sentence})

In [ ]:
pd.DataFrame.from_dict(dmk_sentences).to_excel('./[filtered]greek_news_by_sentences_function.xlsx', index = False)

### With spacy

In [28]:
# !python -m spacy download en_core_web_sm

     --------------------------------------- 12.8/12.8 MB 43.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [29]:
nlp = spacy.load('en_core_web_sm')

In [40]:
dmk_sentences = []
weird_to_remove = ["_x000D_"] #some symbols to manually delete
for idx,row in dmk.iterrows():

    for sentence in nlp(row['paragraph']).sents:
        sent_str=sentence.text
        for remove in weird_to_remove:
            sent_str= re.sub(remove,"",sent_str)
        
        if len(sent_str.split())>2: #just in case any symbol or punctuation was not filtered
            dmk_sentences.append({'title':row['title'], 'date':row['date'], 'sentence':sentence})

In [41]:
pd.DataFrame.from_dict(dmk_sentences).to_excel('./[filtered]greek_news_by_sentences_spacy.xlsx', index = False)

## Paragraphs filtered

In [3]:
dmk['paragraph'] = dmk['paragraph'].apply(functions.text_clean)

In [6]:
pd.DataFrame.from_dict(dmk).to_excel('./[filtered]greek_news_by_paragraphs.xlsx', index = False)